In [2]:
import pyscf
import pyscf.tools
import sys
# git clone https://github.com/nmayhall-vt/OrbitalPartitioning.git
# give path to OrbitalPartitioning repo
sys.path.append('../../../OrbitalPartitioning/orbitalpartitioning')
import orbitalpartitioning
from orbitalpartitioning import *

In [3]:
molecule = '''
C       -6.8604981940     -0.4376683883      0.0000000000
C       -7.0417938838      0.9473143671      0.0000000000
C       -5.5765818320     -0.9841959654      0.0000000000
C       -5.9160648559      1.7742824642      0.0000000000
C       -4.4255614400     -0.1700917199      0.0000000000
C       -4.6336867757      1.2242295397      0.0000000000
C       -3.0514687259     -0.7633813258      0.0000000000
C       -2.8520613255     -2.1570405955      0.0000000000
C       -1.5640941394     -2.6887327507      0.0000000000
C       -0.4451033014     -1.8584342989      0.0000000000
C       -0.5927492072     -0.4581981628      0.0000000000
C       -1.9041039939      0.0505338987      0.0000000000
H       -8.0463585128      1.3756739224      0.0000000000
H       -5.4821758879     -2.0696091478      0.0000000000
H       -6.0325833442      2.8604965477      0.0000000000
H       -3.7849879120      1.9076345469      0.0000000000
H       -3.6983932318     -2.8425398416      0.0000000000
H       -1.4298106521     -3.7726953568      0.0000000000
H        0.5426199813     -2.3168156796      0.0000000000
H       -2.0368540411      1.1280906363      0.0000000000
H       -7.7260669052     -1.1042415323      0.0000000000
H        0.22700           0.16873           0.00000
'''
basis = "cc-pvdz"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        #symmetry=   True,
        spin    =   0, # number of unpaired electrons
        charge  =   0,
        basis   =   basis)


pymol.build()
#print("symmetry: ",pymol.topgroup)
mf = pyscf.scf.RHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
#mf.conv_tol_grad = 1e-5
#mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
mf.analyze()
# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
print("size of MO coeff", C.shape)
ndocc = pymol.nelec[1]
nsing = pymol.nelec[0] - ndocc
nvirt = pymol.mol.nao - ndocc - nsing
print(ndocc)
print(nsing)
print(nvirt)
# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]
np.save("C_scf", C)

nbas = Cdocc.shape[0]



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/td/qpnnxwv93pq0t7bbdkh5rvzr0000gn/T/tmpppy4megv
max_memory 4000 MB (current use 0 MB)
Set gradient conv threshold to 0.0001
init E= -463.937039728682
  HOMO = -0.201399173349765  LUMO = 0.00065918833382099
cycle= 1 E= -460.148732039677  delta_E= 3.79  |g|= 0.563  |ddm|= 4.88
  HOMO = -0.31292892155204  LUMO = 0.0615105158847815
cycle= 2 E= -460.269257271685  delta_E= -0.121  |g|= 0.16  |ddm|=    1
  HOMO = -0.277271190926687  LUMO = 0.100559181707899
cycle= 3 E= -460.277736606206  delta_E= -0.00848  |g|= 0.0783  |ddm|= 0.408
  HOMO = -0.289669476214775  LUMO = 0.0900201444921001
cycle= 4 E= -460.27951161518  delta_E= -0.00178  |g|= 0.0151  |ddm

In [4]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    #print(ao_idx)
    if ao[0] in (0,1,2,3,4,5):
        if ao[2] in ("2p"):
            if ao[3] == "z":
                print(ao)
                frag1.append(ao_idx)
                full.append(ao_idx)
    if ao[0] in (6,7,8,9,10,11):
        if ao[2] in ("2p"):
            if ao[3] == "z":
                frag2.append(ao_idx)
                full.append(ao_idx)  


frags = [frag1, frag2]
print(frags)
print(len(frag1))
print(len(frag2))

(0, 'C', '2p', 'z')
(1, 'C', '2p', 'z')
(2, 'C', '2p', 'z')
(3, 'C', '2p', 'z')
(4, 'C', '2p', 'z')
(5, 'C', '2p', 'z')
[[5, 19, 33, 47, 61, 75], [89, 103, 117, 131, 145, 159]]
6
6


In [5]:
# Define projectors
nbas = Cdocc.shape[0]
X = scipy.linalg.sqrtm(S)
X = np.eye(nbas) 
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])
display(Pfull.shape)

(218, 12)

In [6]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))
display(Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
print(" Should be 1: ", np.linalg.det(Cact.T @ S @ Cact))

 Partition  218 orbitals into a total of   12 orbitals
            Index   Sing. Val. Space       
                0   1.00000000            2*
                1   1.00000000            2*
                2   1.00000000            2*
                3   1.00000000            2*
                4   1.00000000            2*
                5   1.00000000            2*
                6   0.91524412            0*
                7   0.91023129            0*
                8   0.90788521            0*
                9   0.90607946            0*
               10   0.90555587            0*
               11   0.90319379            0*
               12   0.42923301            2
               13   0.42422702            2
               14   0.42310756            2
               15   0.41921885            2
               16   0.41410022            2
               17   0.40289975            2
  SVD active space has the following dimensions:
  Orbital Block    Environment         Active
  

array([[-1.71179287e-16,  4.27395488e-17,  1.62212117e-16, ...,
        -1.46293922e-16, -2.77530984e-17, -7.65855847e-17],
       [-5.73040115e-17,  2.09893551e-16,  1.72926708e-16, ...,
        -3.43271443e-16, -5.32738586e-17, -5.75274797e-17],
       [ 2.63912186e-15, -1.14872390e-15, -3.02457415e-15, ...,
         9.66902418e-16,  1.14079139e-15,  2.12681624e-15],
       ...,
       [-8.13411488e-17, -1.69089094e-17,  1.71573730e-16, ...,
        -7.08096169e-18, -7.79065050e-17, -4.63592939e-17],
       [ 2.15328339e-16,  1.36589067e-16,  2.10054685e-16, ...,
        -6.16834514e-17,  7.69831224e-17, -6.19860666e-17],
       [ 1.54958226e-03, -6.51683915e-03,  1.15484626e-03, ...,
        -6.53734282e-17,  5.26755623e-17,  4.72958075e-17]])

 Should be 1:  0.9999999999999996


In [7]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    # (Of, Sf, Vf), (Oact, Sact, Vact) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
Cfrags = canonicalize(Cfrags, F)

Cact = np.hstack(Cfrags)

print("nick: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])


print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


 Fragment:  [5, 19, 33, 47, 61, 75]
 Partition   12 orbitals into a total of    6 orbitals
            Index   Sing. Val. Space       
                0   0.99999643            2*
                1   0.99994058            2*
                2   0.98685246            2*
                3   0.90700805            0*
                4   0.90607117            0*
                5   0.90440602            0*
                6   0.14736696            0
                7   0.12288367            2
                8   0.01096706            2
                9   0.00988094            0
               10   0.00241800            0
               11   0.00152824            2
  SVD active space has the following dimensions:
  Orbital Block    Environment         Active
              0              3              3
              1              0              0
              2              3              3

 Fragment:  [89, 103, 117, 131, 145, 159]
 Partition   12 orbitals into a total of    6 orbitals

In [8]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
#np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
#np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)

(218, 35)
(218, 12)
(218, 218)
